# Grab safety challenge

Feature engineering:

(a). Here we sort the all records of a ride (bookingID) based on the time sequence (second) and cut them into multiple pieces, where 1 piece covers at maximum consecutive 10 seconds (e.g. 0-10s, 0-20s, etc). 

(b). For the records in each data piece, we do the following processing for each metric.
     
     Speed: avg speed.
     Acceleration(x,y,z): avg and std.
     Gyro(x,y,z): avg and std.
     Bearing: avg.
     Accuracy: avg.

(c). Then we grant a risk mark based on their threshold values (combination of max, min, avg values) to the important metrics of each piece (metrics are selected based on the feature importance analysis of the training model), and calculate the risk mark of that piece (i.e. the risk mark of the consecutive 10s).

(d). Across all pieces of each ride (bookingID), select the maximum risk mark as the final risk estimation of that ride.


Generally speaking, this method can help us grasp the most risky 10s for the measurement of ride safety.

A noticeable advantage of this method lies in that, it can be easily applied to real-time monitoring for the ongoing rides. As every 10s (can be adjusted) will return a risk mark, we may send reminder/warning to the drivers once we detected a mark surpassing the pre-defined safe threshold and push them drive more carefully.

In [ ]:
from collections import defaultdict
import csv
import pandas as pd
import numpy as np


# --------------------------------------------------------------------------- Data pre-processing
rawDataFileList = ['part-00000-e6120af0-10c2-4248-97c4-81baf4304e5c-c000',
                   'part-00001-e6120af0-10c2-4248-97c4-81baf4304e5c-c000',
                   'part-00002-e6120af0-10c2-4248-97c4-81baf4304e5c-c000',
                   'part-00003-e6120af0-10c2-4248-97c4-81baf4304e5c-c000',
                   'part-00004-e6120af0-10c2-4248-97c4-81baf4304e5c-c000',
                   'part-00005-e6120af0-10c2-4248-97c4-81baf4304e5c-c000',
                   'part-00006-e6120af0-10c2-4248-97c4-81baf4304e5c-c000',
                   'part-00007-e6120af0-10c2-4248-97c4-81baf4304e5c-c000',
                   'part-00008-e6120af0-10c2-4248-97c4-81baf4304e5c-c000',
                   'part-00009-e6120af0-10c2-4248-97c4-81baf4304e5c-c000']

dataInfo = defaultdict(dict)
for rawDataFile in rawDataFileList:
    rawData = csv.DictReader(open(rawDataFile + '.csv'))
    for row in rawData:
        bookingID = row['bookingID']
        rowData = [int(float(row['second'])),
                   float(row['Speed']),
                   int(float(row['Bearing'])),
                   float(row['acceleration_x']),
                   float(row['acceleration_y']),
                   float(row['acceleration_z']),
                   float(row['gyro_x']),
                   float(row['gyro_y']),
                   float(row['gyro_z']),
                   int(float(row['Accuracy']))]
        try:
            dataInfo[bookingID].append(rowData)
        except:
            dataInfo[bookingID] = []
            dataInfo[bookingID].append(rowData)
            
            
# --------------------------------------------------------------------------- Feature engineering
label = csv.DictReader(open('labels.csv'))
labelInfo = dict()
for row in label:
    labelInfo[row['bookingID']] = int(row['label'])
    
featureInfo = {}
finalFeature = defaultdict(dict)                # to save the final feature output for each bookingID

count = 0
for key, value in dataInfo.items():             # iterate for each bookingID
    bookingID = key
    highestRiskMark = -10000
    count += 1
    print(count)
    bookingData = sorted(value, key=lambda x: x[0])
    iterations = int(np.floor(len(bookingData) / 10)) + 1

    for i in range(0, iterations, 1):           # iterate for every 10s to get aggregated stats
        try:
            filteredData = bookingData[i * 10: 10]
        except:
            filteredData = bookingData[i * 10:]

        if len(filteredData) <= 3:
            continue

        # --------------------------------------- Save all records of 10s into a list
        speed = [x[1] for x in filteredData]
        accX = [abs(x[3]) for x in filteredData]
        accY = [abs(x[4]) for x in filteredData]
        accZ = [x[5] for x in filteredData]
        gyroX = [abs(x[6]) for x in filteredData]
        gyroY = [abs(x[7]) for x in filteredData]
        gyroZ = [abs(x[8]) for x in filteredData]
        bearing = [x[2] for x in filteredData]
        accuracy = [x[9] for x in filteredData]

        # --------------------------------------- Feature calculation based on threshold values
        avgSpeed = sum(speed)/len(speed) - 5      # 5 is selected as the threshold for avg speed

        avgAccX = sum(accX)/len(accX) - 3
        sigmaAccX = (sum([(x - sum(accX)/len(accX)) ** 2 for x in accX])**0.5 - 2)/2
        avgAccY = sum(accY)/len(accY) - 8
        sigmaAccY = (sum([(x - sum(accY)/len(accY)) ** 2 for x in accY])**0.5 - 2)/2
        avgAccZ = sum(accZ)/len(accZ) - 4
        sigmaAccZ = (sum([(x - sum(accZ)/len(accZ)) ** 2 for x in accZ])**0.5 - 3)/2

        avgGyroX = sum(gyroX)/len(gyroX)/0.5
        sigmaGyroX = ((sum([(x - sum(gyroX)/len(gyroX)) ** 2 for x in gyroX])**0.5)-0.5)/0.5
        avgGyroY = sum(gyroY)/len(gyroY)/0.5
        sigmaGyroY = (sum([(x - sum(gyroY)/len(gyroY)) ** 2 for x in gyroY])**0.5-0.5)/0.5
        avgGyroZ = sum(gyroZ)/len(gyroZ)/0.5
        sigmaGyroZ = (sum([(x - sum(gyroZ)/len(gyroZ)) ** 2 for x in gyroZ])**0.5-0.5)/0.5

        avgBearing = (sum(bearing)/len(bearing)-200)/10
        avgAccuracy = sum(accuracy)/len(accuracy) - 6
        
        # we only consider speed and acceleration relevant metrics, as they are most important in feature importance analysis
        riskMark = avgSpeed + avgAccX + sigmaAccX + avgAccY + sigmaAccY + avgAccZ + sigmaAccZ  

        if riskMark > highestRiskMark:            # if the current 10s has the highest risk mark, use it as the ride feature
            highestRiskMark = riskMark
            featureInfo[bookingID]=[avgSpeed,
                                   avgAccX,
                                   sigmaAccX,
                                   avgAccY,
                                   sigmaAccY,
                                   avgAccZ,
                                   sigmaAccZ,
                                   avgGyroX,
                                   sigmaGyroX,
                                   avgGyroY,
                                   sigmaGyroY,
                                   avgGyroZ,
                                   sigmaGyroZ,
                                   avgBearing,
                                   avgAccuracy]

    # ------------------------------------------- Final feature output for each bookingID
    featureList = ['avgSpeed',
                   'avgAccX',
                   'sigmaAccX',
                   'avgAccY',
                   'sigmaAccY',
                   'avgAccZ',
                   'sigmaAccZ',
                   'avgGyroX',
                   'sigmaGyroX',
                   'avgGyroY',
                   'sigmaGyroY',
                   'avgGyroZ',
                   'sigmaGyroZ',
                   'avgBearing',
                   'avgAccuracy']

    for i in range(0, len(featureList), 1):
        finalFeature[bookingID][featureList[i]] = featureInfo[bookingID][i]

# ----------------------------------------------- Write out the features
header = ['bookingID'] + featureList + ['label']
writer = csv.DictWriter(open('Safety_marked_feature.csv', 'wb'), header)
writer.writeheader()

for bookingID, value in finalFeature.items():
    value['bookingID'] = bookingID
    value['label'] = labelInfo[bookingID]
    writer.writerow(value)

XGBOOST training model.

In [ ]:
# ----------------------------------------------- Data reading
df = pd.read_csv('Safety_marked_feature.csv')

trainSize = int(float(df.shape[0])*0.8)
train = df[: trainSize]
test = df[trainSize :]

featureList = ['avgSpeed',
               'avgAccX',
               'sigmaAccX',
               'avgAccY',
               'sigmaAccY',
               'avgAccZ',
               'sigmaAccZ',
               'avgGyroX',
               'sigmaGyroX',
               'avgGyroY',
               'sigmaGyroY',
               'avgGyroZ',
               'sigmaGyroZ',
               'avgBearing'，
               'avgAccuracy'
               ]

trainX = train[featureList]
trainY = train['label']
testX = test[featureList]
testY = test['label']

# ----------------------------------------------- CV and parameter selection
#xgbModel = xgb.XGBClassifier(objective='rank:pairwise', learning_rate=0.1)
#clf = GridSearchCV(xgbModel,
#                   {'max_depth': [4, 5, 6, 7, 8],
#                    'n_estimators': [300, 500, 800, 1000]},
#                   verbose=1)
#clf.fit(trainX, trainY)
#print(clf.best_score_)
#print(clf.best_params_)

# ----------------------------------------------- Model training and outcome prediction
gbm = xgb.XGBClassifier(max_depth=5,
                        learning_rate=0.05,
                        silent=1,
                        eval_metric='auc',
                        n_estimators=500,
                        objective='rank:pairwise',
                        reg_lambda=0.1
                        )
gbm.fit(trainX, trainY)

predictY = gbm.predict(trainX)
predictY = (predictY >= 0.5) * 1
print 'Training AUC: %.4f' % metrics.roc_auc_score(trainY, predictY)
print 'Training ACC: %.4f' % metrics.accuracy_score(trainY, predictY)

predictY = gbm.predict(testX)
predictY = (predictY >= 0.5) * 1
print 'Testing AUC: %.4f' % metrics.roc_auc_score(testY, predictY)
print 'Testing ACC: %.4f' % metrics.accuracy_score(testY, predictY)

# ----------------------------------------------- Feature importance analysis
#plot_importance(gbm)
#plt.show()